In [89]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import scipy
import math
from scipy import misc
from glob import glob
import json
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 20)

Populating the interactive namespace from numpy and matplotlib


In [8]:
# Load in images *ALL*
PROCESS_ALL_IMAGES = False
dataset = "24-06-2017"

# Read in photos, their labels, then the image, then zip together
image_files = glob("/Users/Alex/Desktop/tagging_iterations/%s/**/%s.jpg" % (dataset, ("*" if PROCESS_ALL_IMAGES else "*362 (6)")))
labels = [json.load((open("%s.json" % p))) for p in image_files]

images = [misc.imread(p) for p in image_files]
image_identifiers = [os.path.splitext(os.path.basename(p))[0] for p in image_files]

# Resize all our images to SCALE%; map coordinates to new scale
SCALE = 0.2
images = [misc.imresize(p, SCALE) for p in images]
data = dict(zip(image_identifiers, images))

# Setup output
outdir = "/Users/Alex/Desktop/augmented_images/%s" % dataset
if not os.path.exists(outdir):
    os.makedirs(outdir)

# TODO: Ignore scaling for now, use imgaug instead to scale down!

# def convert_string_bib_coord_to_ia_keypoint(coord_str):
#     coords = [ int(int(pt) * SCALE) for pt in coord_str.split(', ') ]
#     keypoint = ia.Keypoint(x=coords[0], y=coords[1])
#     ia.KeypointsOnImage(keypoints, shape=image.shape)
#     return keypoint

# def extract_bib_sheet_coords(label):
#     return [np.array([
#         convert_string_bib_coord_to_ia_keypoint(s) for s in runner['Bib']['PixelPoints']
#     ]) for runner in label['TaggedRunners']]

def extract_bib_keypoints_on_image_from_label(label):    
    def extract_bib_keypoint_from_coords_str(coords_str):
        coords = [ int(int(pt) * SCALE) for pt in coords_str.split(', ') ]
        keypoint = ia.Keypoint(x=coords[0], y=coords[1])
        return keypoint
    
    def extract_bib_keypoints_from_runner(runner):
        coords = runner["Bib"]["PixelPoints"]
        return [ extract_bib_keypoint_from_coords_str(c) for c in coords ]
    
    # Extract the image
    image_identifier = label["Identifier"]
    image = data[image_identifier]
    
    # Flatten each runner down
    keypoints = np.array([ extract_bib_keypoints_from_runner(runner) for runner in label["TaggedRunners"] ]).flatten()
    
    # Return a single KeypointsOnImage
    return ia.KeypointsOnImage(keypoints, shape=image.shape)

# Extract all bib sheets and their respective coordinates and map to scaled matrix
keypoints = [ extract_bib_keypoints_on_image_from_label(label) for label in labels ]

In [9]:
def affine():
    TRANSLATE_PCT_RANGE = 0.35
    ROTATION_RANGE = (-45,45)
    SHEAR_RANGE = (-5,5) 
    
    translate_percent = {
        "x": (-TRANSLATE_PCT_RANGE, +TRANSLATE_PCT_RANGE),
        "y": (-TRANSLATE_PCT_RANGE, +TRANSLATE_PCT_RANGE),
    }
    rotate=ROTATION_RANGE
    shear=SHEAR_RANGE
    mode = "edge"
    
    return iaa.Affine(translate_percent=translate_percent,
                      rotate=rotate,
                      shear=shear,
                      mode=mode)

def add_neg():
    return iaa.Add((-45, 0))

def add_pos():
    return iaa.Add((0, 45))

def mul_neg():
    return iaa.Multiply((0.5, 1))

def mul_pos():
    return iaa.Multiply((1, 1.5))

def blur():
    return one_of([
        iaa.GaussianBlur((0, 3.0)),
        iaa.AverageBlur(k=(2, 4)),
        iaa.MedianBlur(k=(3, 5)),
    ])

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
def sometimes(aug, pct = 0.5):
    return iaa.Sometimes(pct, aug)
    
def one_of(funcs):
    return iaa.OneOf(funcs)

seq = iaa.Sequential(
    [
        affine(),
        sometimes(one_of([add_pos(), add_neg()])),
        sometimes(one_of([mul_pos(), mul_neg()])),
        sometimes(blur(), 0.3)
    ],
    random_order=True
)

In [106]:
def keypoints_per_person(kpts):
    # Group one keypoint per person
    return [kpts.keypoints[i:i + 4] for i in range(0, len(kpts.keypoints), 4)]

def plot_img(src_image, src_keypoints, aug_image, aug_keypoints, aug_rects):
    def plot_keypoints_on_ax(kpts, ax_id):
        polys = keypoints_per_person(kpts)
        for poly in polys:
            coords = [[coords.x, coords.y] for coords in poly]
            ax[ax_id].add_patch(patches.Polygon(coords, linewidth=3, edgecolor='lime', fill=False))
    
    fig, ax = plt.subplots(2)
    ax[0].imshow(src_image)
    ax[1].imshow(aug_image)
    plot_keypoints_on_ax(src_keypoints, 0)
    plot_keypoints_on_ax(aug_keypoints, 1)
    
    for rect in aug_rects:
        ax[1].add_patch(patches.Rectangle((rect["x"], rect["y"]), width=rect["w"], height=rect["h"], fill=False, linestyle="dashed", linewidth=3, color="red"))
    
    return fig

def show_results(img_idx):
    plt.close()
    image = images[img_idx]
    image_aug_keypoints = valid_keypoints(aug_keypoints[img_idx], image)
    image_aug_rects = keypoints_to_rects(image_aug_keypoints)
    image_aug_keypoints = ia.KeypointsOnImage(np.array(image_aug_keypoints).flatten(), shape=image.shape)
    return plot_img(images[img_idx], keypoints[img_idx], aug_images[img_idx], image_aug_keypoints, image_aug_rects)

In [112]:
def valid_keypoints(kpts, image):
    # Returns any keypoints that are outside the width/height of the image
    width = image.shape[1]
    height = image.shape[0]
    # Group by four (for each person)
    runner_keypoints = keypoints_per_person(kpts)
    # Copy over the "valid" keypoints (assume all are valid)
    valid_keypoints = [e for e in runner_keypoints]
    for kpts in runner_keypoints:
        for k in kpts:
            # If hidden, remove this runner
            if k.x < 0 or k.x > width or k .y < 0 or k.y > height:
                # Remove from valid if hidden
                valid_keypoints.remove(kpts)
                break
    # Whatever remains becomes the Bib click points for these runners
    return valid_keypoints

def keypoints_to_rects(kpts):
    rects = []
    for kpt in kpts:
        xs = [i.x for i in kpt]
        ys = [i.y for i in kpt]
        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)
        x = min_x
        y = min_y
        w = max_x - min_x
        h = max_y - min_y
        rects.append({"x": x, "y": y, "w": w, "h": h})
    return rects

def write_class_file(image):
    img_idx = images.index(image)
    image_identifier = image_identifiers[img_idx]
    image_aug_keypoints = valid_keypoints(aug_keypoints[img_idx], image)
    image_aug_rects = keypoints_to_rects(image_aug_keypoints)
    lines = []
    for rect in image_aug_rects:
        line = "bib %i %i %i %i" % (rect["x"], rect["y"], rect["w"], rect["h"])
        lines.append(line)
    text_file = open("%s/%s.txt" % (outdir, image_identifier), "w")
    text_file.write(lines.join("\n"))
    text_file.close()
        
if not PROCESS_ALL_IMAGES:
    for i in range(25):
        # Process 25 images
        seq_det = seq.to_deterministic()
        aug_images = seq_det.augment_images(images)
        aug_keypoints = seq_det.augment_keypoints(keypoints)
        show_results(0).savefig("/Users/Alex/Desktop/augmented_images_test/%s_augmented.png" % i)

SyntaxError: invalid syntax (<ipython-input-112-1286a2105066>, line 38)

In [ ]:
w